In [77]:
#########################################
# Toy implementation of factoring algorithm (Algorithm 2.2) in 
# "Factoring using multiplicative relations modulo n:
#      a subexponential factoring algorithm inspired by the index calculus
# by Katherine E. Stange
#########################################

def get_order(n, g, B=10, rels=10, verbose=True, alphas=False, texify=False):
    # INPUT:
    # n = modulus
    # g = base
    # B = bound for primes
    # rels = number of additional relations
    # verbose = whether to print info as you go
    # alphas = whether to return the alpha values
    # textify = whether to print latex for algorithm
    # OUTPUT:  order of g modulo n (with high probability)
    
    # Set up factor base
    if verbose:
        print("Attempting to compute the order of ", g, "modulo", n)
    numPrimes = prime_pi(B)  # the number of primes <= B
    if verbose:
        print("The number of primes in the factor base:", numPrimes)
        print("These are:", [nth_prime(m) for m in range(1, numPrimes+1)])
    
    # Set the number of relations to find
    relsDesired = numPrimes + rels  # the number of relations to find
    if verbose:
        print("We are looking for ", relsDesired, "relations.")
    vectors = []  # to store the relations we find
        
    # Main relation finding loop
    searchCount = 0 # number of smoothness tests run
    relsFound = 0 # number of relations found
    while( relsFound < relsDesired ):
        
        # take a random power of x
        x = randint(1,n)
        residue = ZZ(Mod(g,n)^x)
        
        # trial division to test smoothness
        fac = [0 for _ in range(B+1)]
        for p in primes(B):
            while Mod(residue,p) == 0:
                residue = residue/p
                fac[p] += 1
        
        if residue == 1:  # store a relation if smooth
            if verbose:
                print("found a relation:", g, "^", x, "is", factor(ZZ(Mod(g,n)^x)))
            if texify:
                print(g,"^{",x,"}&=",latex(factor(ZZ(Mod(g,n)^x))),",\\\\")
            vec = vector([fac[nth_prime(m)] for m in range(1,numPrimes+1)])
            vectors.append([vec,x])
            relsFound += 1
            
        searchCount += 1
        
    if verbose:
        print("I searched through ", searchCount, "powers of ", g)
        
    # Linear algebra phase
    relMatrix = matrix([vectors[i][0] for i in range(relsDesired)]).transpose()
    if verbose:
        print("The relation matrix is (cols are relations):")
        print(relMatrix)
    if texify:
        print(latex(relMatrix))
    kernel = relMatrix.right_kernel().basis()
    if verbose:
        print("The right kernel has basis:")
        print(kernel)
    if texify:
        print(latex(matrix(kernel)))
    alphaVals = []
    for basisVec in kernel:
        alpha = sum([basisVec[i]*vectors[i][1] for i in range(relsDesired)])
        alphaVals.append(alpha)
    if verbose:
        print("The corresponding sums of the x's are:")
        print(alphaVals)
    if texify:
        print(latex(alphaVals))
        
    # GCD phase
    finalGCD = 0
    for alpha in alphaVals:
        finalGCD = gcd(finalGCD,alpha)
        
    # Report and return
    if verbose:
        print("Their gcd is:", finalGCD)
        print("******* Relation to reality? ******")
        print("Sage's factorization of", n, "is", factor(n))
        print("The actual multiplicative order is:", Mod(g,n).multiplicative_order())
        if Mod(g,n).multiplicative_order() == finalGCD:
            print("Success!")
        else:
            print("Failure!")
            print("The ratio of the expected value to the real value is (1=success):", finalGCD/Mod(g,n).multiplicative_order())
    if alphas:
        return(finalGCD,[a/finalGCD for a in alphaVals])
    return(finalGCD)
    
            
    

In [78]:
get_order(701*89, g=43, B=50, verbose=True, texify=False)

Attempting to compute the order of  43 modulo 62389
The number of primes in the factor base: 15
These are: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
We are looking for  25 relations.
found a relation: 43 ^ 56280 is 3 * 5^2 * 11 * 13
found a relation: 43 ^ 1162 is 2 * 3 * 17 * 29
found a relation: 43 ^ 10232 is 2^4 * 5^4
found a relation: 43 ^ 23353 is 2 * 3^3 * 5^2 * 17
found a relation: 43 ^ 56552 is 2^2 * 3^2 * 19 * 29
found a relation: 43 ^ 50800 is 2^7 * 3^2 * 47
found a relation: 43 ^ 38196 is 3^3 * 11 * 43
found a relation: 43 ^ 27924 is 3 * 11 * 23 * 47
found a relation: 43 ^ 52568 is 2 * 37 * 41
found a relation: 43 ^ 1813 is 3^2 * 7 * 13^2
found a relation: 43 ^ 15415 is 2^7 * 3^2 * 23
found a relation: 43 ^ 5914 is 3^2 * 7^2 * 17
found a relation: 43 ^ 6148 is 2^3 * 3^3 * 5 * 29
found a relation: 43 ^ 41609 is 5 * 7 * 29^2
found a relation: 43 ^ 24316 is 2 * 3^5 * 7 * 17
found a relation: 43 ^ 28225 is 3 * 7^3 * 23
found a relation: 43 ^ 51425 is 3 * 11 * 29 * 

15400